# Face Recognition

[Face Recognition](https://github.com/ageitgey/face_recognition)

## Installation

### Requirements

  * Python 3.3+ or Python 2.7
  * macOS or Linux (Windows not officially supported, but might work)

### Installation Options:

#### Installing on Mac or Linux

First, make sure you have dlib already installed with Python bindings:

  * [How to install dlib from source on macOS or Ubuntu](https://gist.github.com/ageitgey/629d75c1baac34dfa5ca2a1928a7aeaf)

Then, install this module from pypi using `pip3` (or `pip2` for Python 2):

```bash
pip3 install face_recognition
```

If you are having trouble with installation, you can also try out a
[pre-configured VM](https://medium.com/@ageitgey/try-deep-learning-in-python-now-with-a-fully-pre-configured-vm-1d97d4c3e9b).

#### Installing on Raspberry Pi 2+

  * [Raspberry Pi 2+ installation instructions](https://gist.github.com/ageitgey/1ac8dbe8572f3f533df6269dab35df65)

#### Installing on Windows

While Windows isn't officially supported, helpful users have posted instuctions on how to install this library:

  * [@masoudr's Windows 10 installation guide (dlib + face_recognition)](https://github.com/ageitgey/face_recognition/issues/175#issue-257710508)

#### Installing a pre-configured Virtual Machine image

  * [Download the pre-configured VM image](https://medium.com/@ageitgey/try-deep-learning-in-python-now-with-a-fully-pre-configured-vm-1d97d4c3e9b) (for VMware Player or VirtualBox).

## Before get started

### Create folders
#### known_faces

* Create a folder known_faces and put pictures of known people in this folder. The format of picture can be either jpg or jpeg which I've verified however other formats may be supported as well.
* Naming convention: name the picture with the actual name of underlying people.


In [ ]:
import face_recognition
image = face_recognition.load_image_file("./unknown_pictures/unknown.jpeg")
face_locations = face_recognition.face_locations(image)

# Examples
## Find faces from a picture and 

1. Find faces from a picture
2. Find and lookup faces from a picture and 
3. Take a picture and look it up from existing faces
4. Real time face recognition from video

In [ ]:
from os import listdir
import numpy as np
from PIL import Image, ImageDraw
import face_recognition
from os.path import isfile, join, splitext, split
import cv2

In [ ]:
def known_face_enconding(known_face_path='./known_faces/'):
    face_files = [f for f in listdir(known_face_path) if splitext(f)[1] in ('.jpeg','.jpg')]
    encodings = []
    names = []
    for i, f in enumerate(face_files):
        face_name = splitext(split(f)[1])[0]
        face_image = face_recognition.load_image_file(join(known_face_path, f))
        encodings.append(face_recognition.face_encodings(face_image)[0])
        names.append(face_name) 
    return names, encodings
known_names, known_encodings = known_face_enconding()


# Recognize faces from a picture

In [ ]:
def recognize_faces_from_image(input_image, known_names, known_faces, cutoff=0.55, show_features=False):
    names_recognized = []

    # Load the jpg file into a numpy array
    image = face_recognition.load_image_file(input_image)

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Find all facial features in all the faces in the image
    face_landmarks_list = face_recognition.face_landmarks(image)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)

    print("I found {} face(s) in this photograph.".format(len(face_landmarks_list)))

    # Let's trace out each facial feature in the image with a line!
    pil_image = Image.fromarray(image)
    d = ImageDraw.Draw(pil_image)    

    for (top, right, bottom, left), face_encoding, face_landmarks in zip(face_locations, face_encodings, face_landmarks_list):

        # Print the location of each facial feature in this image
        facial_features = [
            'chin',
            'left_eyebrow',
            'right_eyebrow',
            'nose_bridge',
            'nose_tip',
            'left_eye',
            'right_eye',
            'top_lip',
            'bottom_lip'
        ]

        distances = face_recognition.face_distance(known_encodings,face_encoding)
        min_distance = np.min(distances)
        if min_distance < cutoff:
            face_name = known_names[np.argmin(distances)]
            names_recognized.append(face_name)
        else:
            face_name = 'unknown'
        d.text((left,top ), text=face_name)
        d.rectangle(xy=(left,top, right, bottom))
        if show_features:
            for facial_feature in facial_features:
                d.line(face_landmarks[facial_feature], width=1)
    pil_image.show()
    print ("Faces recognized:", names_recognized)    
    return names_recognized



In [ ]:
known_names, known_encodings = known_face_enconding()
names = recognize_faces_from_image("./unknown_faces/unknown.jpeg", known_names, known_faces)

# Recognize faces from real-time video stream

In [ ]:
# This is a super simple (but slow) example of running face recognition on live video from your webcam.
# There's a second example that's a little more complicated but runs faster.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
def recognize_faces_from_stream(known_names, known_faces, cutoff=0.55):
    video_capture = cv2.VideoCapture(0)
    try:
        while True:
            # Grab a single frame of video
            ret, frame = video_capture.read()

            # Find all the faces and face enqcodings in the frame of video
            face_locations = face_recognition.face_locations(frame)
            face_encodings = face_recognition.face_encodings(frame, face_locations)

            # Loop through each face in this frame of video
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # See if the face is a match for the known face(s)
                distances = face_recognition.face_distance(known_encodings, face_encoding)

                min_distance = np.min(distances)
                if min_distance < 0.6:
                    face_name = known_names[np.argmin(distances)]
                else:
                    face_name = 'unknown'   


                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, face_name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            # Display the resulting image
            cv2.imshow('Video', frame)

            # Hit 'q' on the keyboard to quit!
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Release handle to the webcam
        video_capture.release()
        cv2.destroyAllWindows()
    except:
        video_capture.release()
        cv2.destroyAllWindows()